In [ ]:
#Import Stuff
import thinkplot
import thinkstats2
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Probability Density Functions

Probability density functions (PDFs) are another of our commonly used visualizations to show and explore data. 

Going forward, the PDF is probably the most generally useful of the assorted distribution graphs, especially with continuous data. All of them do the same thing, but as we've seen there are some scenarios where the histogram looks weird and isn't useful. The PDF will pretty much always give us a good idea of the distribution. 

## Discreet and Continuous

The relationship between the discreet and continuous distributions is important because we sometimes need/want to transform our data between the two. 

One example from real life is your GPA - when you do assignments/exams, you end up with a raw percentage grade which is continuous. When this is converted to a letter scale (A, B, etc...), that letter scale is discreet - there's only a selection of possible values (b-,b,b+, etc...). This is binning. We take a continuous varaible and create a discreet variable from it - or more literally, we put the continuous values into a bin. 

The other example is when your GPA is caclulated - those discreet values are assigned numbers on a 1-4 scale, then averaged together creating a new continuous value - your GPA.

One place where this is commonly used is lending and credit scores. Having a credit score of 752 vs 764 makes no difference, you're placed in a category of "excellent", "very good", etc...


In [ ]:
#Load data
df = pd.read_csv("data/loan_data.csv")
df.head()

#### Winning is Binning

Below is an example of how binning works that you can walk through. The logic is fairly simple, we see what bucket our numerical value falls into, then give it a label for that group. Here we are giving credit scores their label as though someone was applying for credit. 

<b>Note:</b> I have commented out some of my troubleshooting print statements, try uncommenting and executing the loop, and take a look at what gets printed. If your loop was not delivering what you want, would this be useful in finding the issue?

In [ ]:
#Create credit-score bucket.
df["grade"] = " "
scoreCol = df.columns.get_loc("fico")
gradeCol = df.columns.get_loc("grade")

for i in range(len(df)) :
    if df.iloc[i,scoreCol] < 580:
        #print("Less than 580-"+str(df.iloc[i,scoreCol]))
        df.iloc[i,gradeCol] = "subprime"
    elif df.iloc[i,scoreCol] < 670:
        #print("580-670-"+str(df.iloc[i,scoreCol]))
        df.iloc[i,gradeCol] = "fair"
    elif df.iloc[i,scoreCol] < 740:
        #print("670-740-"+str(df.iloc[i,scoreCol]))
        df.iloc[i,gradeCol] = "good"
    elif df.iloc[i,scoreCol] < 800:
        #print("740-800-"+str(df.iloc[i,scoreCol]))
        df.iloc[i,gradeCol] = "very good"
    else:
        #print("800+-"+str(df.iloc[i,scoreCol]))
        df.iloc[i,gradeCol] = "excellent"
#print(str(scoreCol)+ " "+ str(gradeCol))
df.head(25)

##### Manual Binning via a Loop - Details

What we have above is an example of using a simple-ish loop, let's look at the parts in detail:
<ul>
<li> The "for" part says, "do what's in this loop once for every index (the position in the dataframe - row 1, 2, 3, 4, etc). Each time the stuff is executed, make the variable 'i' be the row number". 
<li> The series of if statements checks if the fico score value is less than a series of cutoffs. If the fico score meets any of the criteria, it gets that label, the current execution of the loop ends, and we progress to the next row. 
<li> The iloc part is not the general way we want to use a dataframe, it is grabbing data from the df based on the index values - the row and column numbers. The 'i' is the row number, it starts at 0 and increases with each execution of the loop. The scoreCol value is calculated above, it is just the column number of the "fico" column (6 for me). The whole statement effectively grabs the "cell" at that location, like an Excel cell or a battleship hit. 
<li> This repeats, with 'i' incrementing until we reach the end. 
</ul>

#### Built in Binning

In reality, we can use premade functions to do the work for us. Here we can define the cutoff limits and let the computer figure our the rest. 

In [ ]:
#In generic cases, we can automate this:
bins = np.arange(580, 860, 60) #or
bins = np.array([580, 670, 740, 800])
indicies = np.digitize(df["fico"], bins)
groups = df.groupby(indicies)
for i, group in groups:
    print(i, group["fico"].min(), len(group), np.exp(group["log.annual.inc"]).mean())

In [ ]:
#Graph
#hist2 = thinkstats2.Hist(round(df["fico"], -1))
hist2 = thinkstats2.Hist(df["fico"])
pmf2 = thinkstats2.Pmf(df["fico"])
cdf2 = thinkstats2.Cdf(df["fico"])

In [ ]:
#Create graphs
thinkplot.PrePlot(6, rows =2, cols=3)
thinkplot.Hist(hist2)
thinkplot.SubPlot(2)
thinkplot.Pmf(pmf2)
thinkplot.SubPlot(3)
thinkplot.Cdf(cdf2)
thinkplot.SubPlot(4)
thinkstats2.NormalProbabilityPlot(df["fico"])
thinkplot.SubPlot(5)
thinkstats2.NormalProbabilityPlot(np.log(df["fico"]))
thinkplot.SubPlot(6)
pdf = thinkstats2.EstimatedPdf(df["fico"]) #See more below
thinkplot.Pdf(pdf)
thinkplot.Config()

#### Multiple Plots on a Grid

In the examples both above and below we have a grid of multiple graphs. Each one is done in a different way. 

In the thinkplot stuff we have a helper called PrePlot, which we can use to define the size of the grid, along with the subPlot function that can basically "aim" the next chart into the desired slot. 

In the seaborn stuff below, we have a grid using pyplot subplots. Here we define a set of sublots, then in each individual plot we assign it a slot. 

Both of these are built upon the foundations of pyplot (which is part of matplotlib), so there is often, but not always interoperability. The method below is farily generic and will work in many places, with many graphs. Note that (for reasons we really don't care about, it has to do with if the function only provides an "image" of a graph, or an entire "object" of a graph) there are some scenarios that making a grid like this won't work. The easiest way to confirm is to Google, "seaborn [graphtype] subplots" and there will be several examples (usually stackoverflow ones have good explained examples) that can normally be easily adapted. There are a bunch of ways to do this, and a bunch of semi-related garphing libraries and types of graphs, so there isn't a "one true answer". 

In [ ]:
#We can use seaborn as well

plt.rcParams["figure.figsize"] = (20,5) #makes the default size larger. 
#Everything after the comma is optional. 
fig, ax = plt.subplots(1,2)
sns.distplot(df["fico"], kde_kws={"color":"red", "label":"KDE"}, hist_kws={"label":"Data"}, ax=ax[1])
sns.distplot(df["fico"], bins=bins, kde_kws={"color":"red", "label":"KDE"}, hist_kws={"label":"Data"}, ax=ax[0])
plt.show()

### KDE and Histogram

As above - the KDE produces a smoothed function, and approximates the distribution of the histogram. Especially when the volume of data becomes large, a KDE may be a more useful visualization of a distribution. 

The smaller those bins get, the closer of an approximation. The smoothing factor accounts for 'noise' - e.g. around 750ish. On the whole, this delivers a distribution shape that isn't impacted directly by sharp ups and downs at certain points on the distribution, like a histogram is. For generalizing from a sample to a population, this is useful...


### Skewness

![Skew](images/skew.png)

Skewness. We can visually see the skew - this one is right skewed a bit - the right side is "stretched" out a bit more. Skew is both easy to see and easy to calculate, we also have a quick rule of thumb that we can figure out with basic satistics:
<ul>
<li>If the mean is greater than the median, the distribution is positively skewed.
<li>If the mean is less than the median, the distribution is negatively skewed.
</ul>

The value of the skew can be calculated with a function called Pearson Median Skewness, which is defined as:

$$ Skew = \frac{3(X_i - \mu)}{\sigma} $$

The values for skew don't have any specific meaning, the larger magnitude it is, the larger the skew. As a rule of thumb, values greater than 1 are highly skewed. 

We can verify with caclculations...

In [ ]:
#Skew
skw = thinkstats2.PearsonMedianSkewness(df["fico"])
print(df["fico"].mean())
print(df["fico"].median())
print(skw)

We can show it a little more clearly on the graph by adding some reference lines for mean and median. We can see that the mean is now greater than the median. 

In [ ]:
thinkplot.PrePlot(1)
thinkplot.Pdf(pdf)
thinkplot.axvline(df["fico"].mean(), color="Red", label="Mean")
thinkplot.axvline(df["fico"].median(), color="Green", label="Median")
thinkplot.Config()

## Income

We can explore a different varaible similarly - income. 

We are given the income in log format. Why might that be? Can you investigate a little, and add normal income to the dataframe?

In [ ]:
#create a new column - income. This should show the regular income, not log transformed. 
df["income"] = np.exp(df["log.annual.inc"])
df["income"].describe()

### Exercise #1 - Create a Function

Try to make a function that prints all the 6 graphs we made above when provided with data. 

In [ ]:
# Challenge - try to create a function that makes the suite of 6 graphs above.
# Note: you can print things in the function, this one doesn't need to return anything. 
# Note 2: this function shouldn't reference anything outside of its arguments, and imported libraries. 
#         One of the big things that makes functions useful is that we can use the function in other programs
#         and it won't "need" anything else to function. Like how we can use np.mean(data) with anyhing in the data part. 
def bigGraph(df_in, columnName):

    # Fill Me!!!!!!!!!!!!!!!!!!!!

    return

In [ ]:
# Draw graphs


### Exercise #2: Establish Tax Rates

Try to use the data - break into groups of marginal tax rates:
<ul>
<li>15% on the first $49,020 of taxable income, plus
<li>20.5% on the next $49,020 of taxable income (on the portion of taxable income over 49,020 up to $98,040), plus
<li>26% on the next $53,939 of taxable income (on the portion of taxable income over $98,040 up to $151,978), plus
<li>29% on the next $64,533 of taxable income (on the portion of taxable income over 151,978 up to $216,511), plus
<li>33% of taxable income over $216,511
</ul>

<b>Generate bins for each tax bracket</b>

In [ ]:
# Generate bins for each tax bracket


In [ ]:
#Create the 6 graph set of graphs for original log income


<b>Use the data to estimate the number of people in each tax bracket</b>

Try to look through the bins, if you can. See the solution for a hint, if needed. 

In [ ]:
# count numbers in each bracket

<b>Use the cdf to estimate the number of people who earn Teacher Money - lowest: 59,357, highest: 101,162</b>

In [ ]:
# estimate number of people


<b>Create a PDF showing the distributiion of income. Try both log income, and raw income. </b>

In [ ]:
# Graph


## Exercise #2: Tax Function

Try this! 

In [ ]:
#Challenge - Create a function that takes an income and returns a tax bill, and marginal tax rate:
def muhTaxes(income):
    
    return taxbill, margRate

In [ ]:
# When calling that function, you can do something like this to get the results:
# tax_bill, marginal_rate = muhTaxes(amount_i_earned)
# print("Tax bill: ", tax_bill)
# print("Marginal rate:", marginal_rate)

## So... Why a Log?

Think quickly - we have both the log and non-log data (there's just a math operation that differs between them, so we can always translate - or transform - back and forth), why bother with the log? 

We are ultimately concerned with using our data to draw inference, which means we are introducing some assumptions. Suppose we want to use the data to estimate the distance between the lowest and highest quintiles of income. If we only care about the sample's data, we can just count, no transformation needed. If we are attempting to generalize from our sample to the population (as we will soon), using the normal distribution allows us to use all of the things that are built to deal with normal distributions - in this case slicing segments of the data. Similarly, when we want to do tests to measure things about our data (such as hypothesis tests), many of those tests are created for normal data specifically. 

In general, we want to make data "as easy as possible to analyze", and making data normal (or sometimes transforming it in different ways, like normalization), often makes it easier. So we do it. 